In [82]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

### 1. 데이터 가져오기

In [84]:
site = 'http://book.interpark.com'
url = site + '/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=bgnb_mn&bid2=LiveRanking&bid3=main&bid4='
res = requests.get(url)
html = res.text

In [53]:
soup = BeautifulSoup(html, 'html.parser')

In [54]:
lis = soup.select_one('div.rankBestContentList').find_all('li')
len(lis)

67

## 2. 찾으려고 하는 데이터 찾기

In [58]:
lis = soup.select('div.rankBestContentList > ol > li')
len(lis)

15

### 3. 하나의 데이터로 원하는 정보 추출

In [59]:
li =  lis[0]
href = li.select_one('.coverImage').find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [85]:
title = li.select_one('.itemName').get_text().strip()
title

'미드나잇 라이브러리(25만부 기념 퍼플 에디션)'

In [86]:
author = li.select_one('.author').get_text().strip()
author

'매트 헤이그/노진선 역'

In [87]:
company = li.select_one('.company').get_text().strip()
company

'인플루엔셜'

In [88]:
price = li.select_one('.price > em'). get_text().strip()
price = int(price.replace(',',''))
price

14220

- 1 ~ 9위

In [89]:
li.select_one('.rankBtn_crtl')

In [61]:
li.select_one('.rankBtn_ctrl')['class']

['rankBtn_ctrl', 'rkNum_B01']

In [62]:
li.select_one('.rankBtn_ctrl')['class'][1]

'rkNum_B01'

In [90]:
li.select_one('.rankBtn_ctrl')['class'][1][-1]

'1'

In [63]:
rank = li.select_one('.rankBtn_ctrl')['class']
rank = int(rank[1][-1:])
rank

1

- 10 ~ 15위

In [64]:
li = lis[12]
li.select('.rankBtn_ctrl')

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M02"></span>]

In [92]:
li.select('.rankBtn_ctrl')[0]['class']

['rankBtn_ctrl', 'rkNum_M01']

In [93]:
li.select('.rankBtn_ctrl')[0]['class'][1][-1]

'1'

In [94]:
li.select('.rankBtn_ctrl')[1]['class'][1][-1]

'5'

- 모든 경우에 대처

In [95]:
if len(li.select('.rankBtn_ctrl')) == 1:
    rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
    rank = int(rank)
else:
    rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
    rank += li.select('.rankBtn_ctrl')[1]['class'][1][-1]
    rank = int(rank)
rank

15

- Sub page

In [96]:
url = site +href
res = requests.get(url)
sub_soup = BeautifulSoup(res.text, 'html.parser')

In [97]:
sub_lis = sub_soup.select('ul.bInfo_txt > li')
len(sub_lis)

6

In [98]:
sub_lis[-3].get_text()

'발행 : 2021년 04월 28일'

In [99]:
date = sub_lis[-3].get_text().split(':')[1].strip()
date

'2021년 04월 28일'

In [100]:
import re
date = re.sub('[^0-9]','',date)
date

'20210428'

In [101]:
page = sub_lis[-2].get_text().split(':')[1].strip()
page

'408'

### 4. 페이지에 있는 모든 데이터 추출

In [105]:
lines = []
for li in lis:
    href = li.select_one('.coverImage').find('a')['href']
    title = li.select_one('.itemName').get_text().strip()
    author = li.select_one('.author').get_text().strip()
    company = li.select_one('.company').get_text().strip()
    price = li.select_one('.price > em'). get_text().strip()
    price = int(price.replace(',',''))
    if len(li.select('.rankBtn_ctrl')) == 1:
        rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
        rank = int(rank)
    else:
        rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
        rank += li.select('.rankBtn_ctrl')[1]['class'][1][-1]
        rank = int(rank)
    url = site +href
    res = requests.get(url)
    sub_soup = BeautifulSoup(res.text,'html.parser')
    sub_lis = sub_soup.select('ul.bInfo_txt > li')
    date = sub_lis[-3].get_text().split(':')[1].strip()
    date = re.sub('[^0-9]','',date)
    page = int(sub_lis[-2].get_text().split(':')[1].strip())

    lines.append([rank, title, author, company, price, date[2:], page])


In [106]:
df = pd.DataFrame(lines, columns=['No','제목','저자','출판사','가격','발행일','쪽수'])
df

,No,제목,저자,출판사,가격,발행일,쪽수
0,1,트렌드 코리아 2022,"김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영",미래의창,16200,211006,452
1,2,2022 현직교사들이 들려주는 면접레시피,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,42300,211112,1104
2,3,아직 오지 않은 날들을 위하여,파스칼 브뤼크네르/이세진 역,인플루엔셜,14400,211112,320
3,4,거인의 포트폴리오,강환국,페이지2북스,16200,211108,404
4,5,불편한 편의점(15만부 기념 윈터 에디션),김호연,나무옆의자,12600,210420,268
5,6,거꾸로 읽는 세계사,유시민(柳時敏),돌베개,15750,211029,404
6,6,달러구트 꿈 백화점,이미예,팩토리나인,12420,200708,300
7,8,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),소윤,북로망스,13500,210929,274
8,9,달러구트 꿈 백화점. 2,이미예,팩토리나인,12420,210727,308
9,10,오십에 읽는 논어,최종엽,유노북스,14400,211103,300


In [107]:
df.to_csv('인터파크_베스트셀러(211207).csv', index=False)